In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np

from datetime import timedelta, datetime, timezone

from typing_extensions import Optional

from data_display.first_stage_optimization_plots import *
from data_federation.input_model import SmallflexInputSchema
from utility.pyomo_preprocessing import *
from config import settings

import plotly.express as px
from optimization.input_data_prepocessing import *
COLORS = px.colors.qualitative.Plotly



In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = "/home/ltomasini/gruobi_license/gurobi.lic"


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))

small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()\
.duckdb_to_schema(file_path=output_file_names["duckdb_input"])

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:01<00:00,  7.43it/s]


In [4]:

nb_days = 1
year = 2020
min_datetime = datetime(year, 1, 1, tzinfo=timezone.utc)
max_datetime = datetime(year + 1, 1, 1, tzinfo=timezone.utc)
market_country = "CH"
market = "DA"

hydro_power_mask: pl.Expr = c("name").is_in(["Aegina hydro"])

first_time_delta=timedelta(days=nb_days)

index : dict[str, pl.DataFrame]= generate_baseline_index(
    small_flex_input_schema=small_flex_input_schema,
    year=year,
    first_time_delta=first_time_delta, 
    hydro_power_mask=hydro_power_mask)
water_flow_factor: pl.DataFrame = generate_water_flow_factor(index=index)
basin_volume_table: dict[int, Optional[pl.DataFrame]] = generate_basin_volume_table(
    small_flex_input_schema=small_flex_input_schema, index=index, d_height=1)
power_performance_table = clean_hydro_power_performance_table(
    small_flex_input_schema= small_flex_input_schema, index=index, 
    basin_volume_table=basin_volume_table
    )

basin_volume_table_2: dict[int, Optional[pl.DataFrame]] = generate_basin_volume_table(
    small_flex_input_schema=small_flex_input_schema, index=index, d_height=0.01)
power_performance_table_2 = clean_hydro_power_performance_table(
    small_flex_input_schema= small_flex_input_schema, index=index, 
    basin_volume_table=basin_volume_table_2
    )


In [ ]:
y_col_list: list[str] = ["alpha_turbined", "alpha_pumped"]
x_col="volume"
power_performance: pl.DataFrame = power_performance_table[0]["power_performance"] 
df: pl.DataFrame = calculate_curviness(data=power_performance, x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()
power_performance_2: pl.DataFrame = power_performance_table_2[0]["power_performance"] 

df: pl.DataFrame = calculate_curviness(data=power_performance_2, x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()

df: pl.DataFrame = calculate_curviness(data=power_performance.filter(c("volume") < 3e6), x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()

df: pl.DataFrame = calculate_curviness(data=power_performance.filter((c("volume")  > 0.8e7).__and__(c("volume") < 1.2e7)), x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()

In [ ]:
x_col = "flow"
y_col_list = ["alpha"]

uuid = small_flex_input_schema.hydro_power_plant.filter(c("name") == "Merezenbach")["uuid"][0]

power_performance_merez = small_flex_input_schema.hydro_power_performance_table.filter(c("power_plant_fk")== uuid).with_columns(
    (c("electrical_power")/ c("flow")).alias("alpha")
)

df: pl.DataFrame = calculate_curviness(data=power_performance_merez, x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()

df: pl.DataFrame = calculate_curviness(data=power_performance_merez.filter(c("flow")<0.15), x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()

In [47]:
x_col = "flow"
y_col_list = ["alpha"]

uuid = small_flex_input_schema.hydro_power_plant.filter(c("name") == "Morel")["uuid"][0]

power_performance_morel = small_flex_input_schema.hydro_power_performance_table.filter(c("power_plant_fk")== uuid).with_columns(
    (c("electrical_power")/ c("flow")).alias("alpha")
).filter(c("head") > 245)


df: pl.DataFrame = calculate_curviness(data=power_performance_morel, x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()

df: pl.DataFrame = calculate_curviness(data=power_performance_morel.filter(c("flow")<3), x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()

df: pl.DataFrame = calculate_curviness(data=power_performance_morel.filter(c("flow")>15), x_col=x_col, y_col_list=y_col_list)
fig = plot_curviness_results(df=df, x_col=x_col, y_col_list=y_col_list)
fig.show()

In [9]:
test = pl.DataFrame({"x": range(-1000,  1000)}).with_columns(
    (3*c("x")**2 - 2*c("x") + 1).alias("y")
)
df: pl.DataFrame = calculate_curviness(data=test, x_col="x", y_col_list=["y"])
fig = plot_curviness_results(df=df, x_col="x", y_col_list=["y"])
fig.show()

In [10]:
test = pl.DataFrame({"x": range(-200,  200)}).with_columns(
    (0.01*c("x")**2 - 2*c("x") + 1).alias("y")
)
df: pl.DataFrame = calculate_curviness(data=test, x_col="x", y_col_list=["y"])
fig = plot_curviness_results(df=df, x_col="x", y_col_list=["y"])
fig.show()

In [11]:
test = pl.DataFrame({"x": np.arange(0,  8, 0.1)}).with_columns(
    (3*np.exp(c("x"))).alias("y")
)
df: pl.DataFrame = calculate_curviness(data=test, x_col="x", y_col_list=["y"])
fig = plot_curviness_results(df=df, x_col="x", y_col_list=["y"])
fig.show()

In [12]:
test = pl.DataFrame({"x": np.arange(6,  8, 0.001)}).with_columns(
    (np.exp(c("x"))).alias("y")
)
df: pl.DataFrame = calculate_curviness(data=test, x_col="x", y_col_list=["y"])
fig = plot_curviness_results(df=df, x_col="x", y_col_list=["y"])
fig.show()